In [1]:
import numpy as np 
import pandas as pd 
import os, datetime 
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold 
from tqdm import tqdm
import random 
import time
import pywt
from keras import backend as K

In [2]:
pip install tsaug

In [3]:
import tsaug
from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse 

## Load Dataset

In [4]:
## Load data 
train_features = pd.read_csv('drive/MyDrive/movement_detection/train_features.csv')
train_labels = pd.read_csv('drive/MyDrive/movement_detection/train_labels.csv')
test_features = pd.read_csv('drive/MyDrive/movement_detection/test_features.csv')
sample_submission = pd.read_csv('drive/MyDrive/movement_detection/sample_submission.csv')

In [5]:
X = tf.reshape(np.array(train_features.iloc[:,2:]),[-1, 600, 6])
X = np.asarray(X) 
X.shape

(3125, 600, 6)

In [6]:
y = train_labels['label'].values
y.shape

(3125,)

### Define Model

In [7]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()


In [8]:
def build_model(seq_len, features): 
  inputs=Input((seq_len,features))
  bn = BatchNormalization()(inputs)
  gru = GRU(128,return_sequences=True)(bn)
  dropout = Dropout(0.25)(gru) 
  att_out=attention()(dropout)
  outputs=Dense(61,activation='softmax')(att_out)
  model=Model(inputs,outputs)
  model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 
  return model


In [11]:
model = build_model(600, 18) 
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 600, 18)]         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 600, 18)           72        
_________________________________________________________________
gru_1 (GRU)                  (None, 600, 128)          56832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 600, 128)          0         
_________________________________________________________________
attention_1 (attention)      (None, 128)               728       
_________________________________________________________________
dense_2 (Dense)              (None, 61)                7869      
Total params: 65,501
Trainable params: 65,465
Non-trainable params: 36
______________________________________________________

In [12]:
kfold = StratifiedKFold(n_splits = 10, random_state = 990101, shuffle = True)
for idx,(train_idx, val_idx) in enumerate(kfold.split(X,y)):   
    print("... Validating on fold {} ...".format(idx+1))  
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx] 
    
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = [] 

    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True): 
        for j in range(10): # add random shift 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i]) 
        for j in range(10): # add noise 
            noised = np.random.normal(0, 1, X_train[i].shape) + X_train[i] 
            X_augmented.append(noised) 
            y_augmented.append(y_train[i])   
    

    X_cropped = Crop(random.randint(300, 580), resize = 600).augment(X_train) 
    X_warped = TimeWarp(random.randint(3,20)).augment(X_train)  
    X_reversed = Reverse().augment(X_train) 
    X_quantized = Quantize(random.randint(10,100)).augment(X_train)
    x_dropout = tsaug.Dropout(p=(0.1,0.5)).augment(X_train)

    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)    

    X_train = np.concatenate([X_train, X_augmented, X_cropped, X_warped, X_reversed, X_quantized, x_dropout]) 
    y_train = np.concatenate([y_train, y_augmented, y_train, y_train, y_train, y_train, y_train])  

    print("Train shapes")
    print(X_train.shape, y_train.shape)  

    ###### feature engineering data ##### 
    print("... DFFT Feature Engineering ...")
    X_fourier_real = [] 
    X_fourier_imag = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
        real_part = np.fft.fft(X_train[i]).real 
        imag_part = np.fft.fft(X_train[i]).imag 
        X_fourier_real.append(real_part)
        X_fourier_imag.append(imag_part) 
    
    X_fourier_real = np.asarray(X_fourier_real)  
    X_fourier_imag = np.asarray(X_fourier_imag)
    
    X_val_fourier_real = [] 
    X_val_fourier_imag = [] 
    for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
        real_part = np.fft.fft(X_val[i]).real 
        imag_part = np.fft.fft(X_val[i]).imag 
        X_val_fourier_real.append(real_part) 
        X_val_fourier_imag.append(imag_part)
    
    X_val_fourier_real = np.asarray(X_val_fourier_real) 
    X_val_fourier_imag = np.asarray(X_val_fourier_imag)
    
    X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)  
    X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)

    ##### train model #####  
    print("... Building Model ...")
    # we have 18 features after feature engineering 
    model = build_model(600, 18) 
    print("... Training ...") 
    model_path = 'drive/MyDrive/movement_detection/kfold' + str(idx+1) + '/GRU_ATTN_epoch_{epoch:03d}_val_{val_loss:.3f}_accuracy_{val_accuracy:.3f}.h5'
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 1, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32, # hyperparameter  
              validation_data = (X_val, y_val), 
              callbacks = [learning_rate_reduction, checkpoint, early_stopping])



  2%|▏         | 58/2812 [00:00<00:04, 573.11it/s]

... Validating on fold 1 ...
... Augmenting Data ...


  1%|▏         | 1012/73112 [00:00<00:07, 10115.93it/s]

Train shapes
(73112, 600, 6) (73112,)
... DFFT Feature Engineering ...


100%|██████████| 313/313 [00:00<00:00, 11018.00it/s]


...

... Augmenting Data ...


  1%|          | 908/73138 [00:00<00:07, 9076.59it/s]

Train shapes
(73138, 600, 6) (73138,)
... DFFT Feature Engineering ...


100%|██████████| 312/312 [00:00<00:00, 10458.61it/s]


... Building Model ...
... Training ...
Epoch 1/200
2286/2286 [==============================] - 55s 23ms/step - loss: 2.1388 - accuracy: 0.5398 - val_loss: 1.2096 - val_accuracy: 0.6667

Epoch 00001: val_loss improved from inf to 1.20962, saving model to drive/MyDrive/movement_detection/kfold10/GRU_ATTN_epoch_001_val_1.210_accuracy_0.667.h5
Epoch 2/200
2286/2286 [==============================] - 54s 23ms/step - loss: 1.2718 - accuracy: 0.6575 - val_loss: 1.2046 - val_accuracy: 0.7051

Epoch 00002: val_loss improved from 1.20962 to 1.20459, saving model to drive/MyDrive/movement_detection/kfold10/GRU_ATTN_epoch_002_val_1.205_accuracy_0.705.h5
Epoch 3/200
2286/2286 [==============================] - 53s 23ms/step - loss: 1.2294 - accuracy: 0.6652 - val_loss: 1.0316 - val_accuracy: 0.7340

Epoch 00003: val_loss improved from 1.20459 to 1.03164, saving model to drive/MyDrive/movement_detection/kfold10/GRU_ATTN_epoch_003_val_1.032_accuracy_0.734.h5
Epoch 4/200
2286/2286 [=================

### Make Prediction

In [9]:
## Make prediction
model1 = load_model('drive/MyDrive/movement_detection/kfold1/GRU_ATTN_epoch_005_val_0.944_accuracy_0.744.h5',
                    custom_objects={'attention': attention})
model2 = load_model('drive/MyDrive/movement_detection/kfold2/GRU_ATTN_epoch_009_val_0.950_accuracy_0.767.h5',
                    custom_objects={'attention': attention}) 
model3 = load_model('drive/MyDrive/movement_detection/kfold3/GRU_ATTN_epoch_011_val_1.001_accuracy_0.764.h5',
                    custom_objects={'attention': attention})  
model4 = load_model('drive/MyDrive/movement_detection/kfold4/GRU_ATTN_epoch_018_val_0.832_accuracy_0.792.h5',
                    custom_objects={'attention': attention}) 
model5 = load_model('drive/MyDrive/movement_detection/kfold5/GRU_ATTN_epoch_006_val_0.914_accuracy_0.735.h5',
                    custom_objects={'attention': attention})
model6 = load_model('drive/MyDrive/movement_detection/kfold6/GRU_ATTN_epoch_008_val_0.899_accuracy_0.744.h5',
                    custom_objects={'attention': attention})
model7 = load_model('drive/MyDrive/movement_detection/kfold7/GRU_ATTN_epoch_007_val_0.988_accuracy_0.740.h5',
                    custom_objects={'attention': attention})
model8 = load_model('drive/MyDrive/movement_detection/kfold8/GRU_ATTN_epoch_005_val_0.846_accuracy_0.769.h5',
                    custom_objects={'attention': attention})
model9 = load_model('drive/MyDrive/movement_detection/kfold9/GRU_ATTN_epoch_004_val_0.885_accuracy_0.756.h5',
                    custom_objects={'attention': attention}) 
model10 = load_model('drive/MyDrive/movement_detection/kfold10/GRU_ATTN_epoch_014_val_0.827_accuracy_0.769.h5',
                     custom_objects={'attention': attention}) 


In [10]:
X_test = tf.reshape(np.array(test_features.iloc[:,2:]),[-1, 600, 6])
X_test = np.asarray(X_test)

##### feature engineering for test dataset ##### 
X_test_fourier_real = [] 
X_test_fourier_imag = [] 
for i in tqdm(range(X_test.shape[0]), position = 0, leave = True):
    real_part = np.fft.fft(X_test[i]).real 
    imag_part = np.fft.fft(X_test[i]).imag 
    X_test_fourier_real.append(real_part) 
    X_test_fourier_imag.append(imag_part)
    
X_test_fourier_real = np.asarray(X_test_fourier_real) 
X_test_fourier_imag = np.asarray(X_test_fourier_imag)
X_test = np.concatenate([X_test, X_test_fourier_real, X_test_fourier_imag], axis = 2)  

print(X_test.shape)


100%|██████████| 782/782 [00:00<00:00, 8197.43it/s]

(782, 600, 18)


In [11]:
p1 = model1.predict(X_test) 
p2 = model2.predict(X_test)
p3 = model3.predict(X_test) 
p4 = model4.predict(X_test) 
p5 = model5.predict(X_test) 
p6 = model6.predict(X_test) 
p7 = model7.predict(X_test) 
p8 = model8.predict(X_test)
p9 = model9.predict(X_test)
p10 = model10.predict(X_test)


In [12]:
p_avg = (p1 + p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9 + p10)/10.0

sample_submission.iloc[:,1:] = p_avg

sample_submission.to_csv("drive/MyDrive/movement_detection/GRU_with_attention.csv",index=False)
